In [1]:
import pandas as pd

In [ ]:
import pandas as pd

labevents_df = pd.read_csv(r"C:\Users\aditya\Downloads\readmission\labevents_202208161605.csv")

units_to_keep = ['ng/mL', 'mg/dL', 'mmol/L', 'mEq/L', 'IU/L', 'mm Hg', 'sec']

labevents_filtered = labevents_df[labevents_df['valueuom'].isin(units_to_keep)]

labevents_filtered['flag'] = labevents_filtered['flag'].fillna('normal')
labevents_filtered['flag'] = labevents_filtered['flag'].apply(lambda x: 'normal' if x != 'abnormal' else 'abnormal')

labevents_filtered['flag'] = labevents_filtered['flag'].map({'normal': 0, 'abnormal': 1})

labevents_filtered = labevents_filtered[['row_id', 'subject_id', 'hadm_id', 'itemid', 'valuenum', 'flag']]

labevents_filtered.to_csv(r'C:\Users\aditya\Downloads\readmission\mywork\cleanednew\labevents_modified.csv', index=False)

print("Modified labevents table has been saved to 'labevents_modified.csv'")

In [ ]:
import pandas as pd

admissions_df = pd.read_csv(r"C:\Users\aditya\Downloads\readmission\admissions_202208161605.csv")

admissions_df = admissions_df[admissions_df['hospital_expire_flag'] == 0]

admissions_df['admittime'] = pd.to_datetime(admissions_df['admittime'])
admissions_df['dischtime'] = pd.to_datetime(admissions_df['dischtime'])

admissions_df['length_of_stay'] = (admissions_df['dischtime'] - admissions_df['admittime']).dt.days

relevant_columns = [
    'subject_id', 'hadm_id', 'admittime', 'dischtime', 
    'admission_location', 'discharge_location', 'hospital_expire_flag', 'length_of_stay'
]
admissions_filtered = admissions_df[relevant_columns]

admissions_filtered.to_csv(r"C:\Users\aditya\Downloads\readmission\mywork\cleanednew\admissions_filtered.csv", index=False)

print("Modified admissions table (patients alive) has been saved to 'C:\\Users\\aditya\\Downloads\\readmission\\mywork\\cleanednew\\admissions_filtered.csv'")

In [2]:
diagnoses_df = pd.read_csv(r"C:\Users\aditya\Downloads\readmission\mywork\cleanednew\procedures_icd_202208161605.csv")

In [6]:
diagnoses_df=diagnoses_df.drop(columns=["seq_num"],axis=1)

In [7]:
diagnoses_df

,row_id,subject_id,hadm_id,icd9_code
0,944,62641,154460,3404
1,945,2592,130856,9671
2,946,2592,130856,3893
3,947,55357,119355,9672
4,948,55357,119355,331
...,...,...,...,...
240090,228330,67415,150871,3736
240091,228331,67415,150871,3893
240092,228332,67415,150871,8872
240093,228333,67415,150871,3893


In [9]:
diagnoses_df.to_csv(r'C:\Users\aditya\Downloads\readmission\mywork\cleanednew\procedures_icd_modified2.csv', index=False)

In [10]:
diagnoses_df

,row_id,subject_id,hadm_id,icd9_code
0,944,62641,154460,3404
1,945,2592,130856,9671
2,946,2592,130856,3893
3,947,55357,119355,9672
4,948,55357,119355,331
...,...,...,...,...
240090,228330,67415,150871,3736
240091,228331,67415,150871,3893
240092,228332,67415,150871,8872
240093,228333,67415,150871,3893


In [18]:
diagnoses = pd.read_csv(r"C:\Users\aditya\Downloads\readmission\diagnoses_icd_202208161605.csv")
diagnoses

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [19]:
diagnoses=diagnoses.drop(columns=["seq_num"],axis=1)

In [20]:
diagnoses.to_csv(r"C:\Users\aditya\Downloads\readmission\mywork\cleanednew\diagnoses_icd_modified2.csv")

In [ ]:
diagnoses_df.to_csv(r'C:\Users\aditya\Downloads\readmission\mywork\cleanednew\procedures_icd_modified2.csv', index=False)

In [ ]:
import pandas as pd

diagnoses_icd = pd.read_csv(r"C:\Users\aditya\Downloads\readmission\mywork\cleanednew\diagnoses_icd_modified2.csv")
admissions = pd.read_csv(r"C:\Users\aditya\Downloads\readmission\mywork\cleanednew\admissions_filtered.csv")
patients = pd.read_csv(r"C:\Users\aditya\Downloads\readmission\mywork\cleanednew\patients_modified.csv")
labevents = pd.read_csv(r"C:\Users\aditya\Downloads\readmission\mywork\cleanednew\labevents_modified.csv")
procedures_icd = pd.read_csv(r'C:\Users\aditya\Downloads\readmission\mywork\cleanednew\procedures_icd_modified2.csv')

adm_diagnoses = pd.merge(
    diagnoses_icd,
    admissions,
    on=['subject_id', 'hadm_id'],
    how='inner'
)

adm_diagnoses_patients = pd.merge(
    adm_diagnoses,
    patients,
    on='subject_id',
    how='inner'
)

lab_agg = labevents.groupby(['subject_id', 'hadm_id']).agg(
    abnormal_lab_count=('flag', 'sum'),
    lab_test_count=('flag', 'count')
).reset_index()

dataset_with_labs = pd.merge(
    adm_diagnoses_patients,
    lab_agg,
    on=['subject_id', 'hadm_id'],
    how='left'
)
dataset_with_labs['abnormal_lab_count'] = dataset_with_labs['abnormal_lab_count'].fillna(0)
dataset_with_labs['lab_test_count'] = dataset_with_labs['lab_test_count'].fillna(0)

proc_agg = procedures_icd.groupby(['subject_id', 'hadm_id']).agg(
    procedure_count=('icd9_code', 'count')
).reset_index()

final_dataset = pd.merge(
    dataset_with_labs,
    proc_agg,
    on=['subject_id', 'hadm_id'],
    how='left'
)

final_dataset['procedure_count'] = final_dataset['procedure_count'].fillna(0)

admissions['admittime'] = pd.to_datetime(admissions['admittime'])
admissions['dischtime'] = pd.to_datetime(admissions['dischtime'])

admissions = admissions.sort_values(['subject_id', 'admittime'])

admissions['next_admittime'] = admissions.groupby('subject_id')['admittime'].shift(-1)
admissions['days_to_next_admit'] = (admissions['next_admittime'] - admissions['dischtime']).dt.days

admissions['readmitted_30_days'] = (admissions['days_to_next_admit'] <= 30) & (admissions['days_to_next_admit'].notna())
admissions['readmitted_30_days'] = admissions['readmitted_30_days'].astype(int)

final_dataset = pd.merge(
    final_dataset,
    admissions[['subject_id', 'hadm_id', 'readmitted_30_days']],
    on=['subject_id', 'hadm_id'],
    how='left'
)

final_dataset.to_csv(r"C:\Users\aditya\Downloads\readmission\mywork\cleanednew\final_dataset3.csv", index=False)
print("Final dataset has been saved to 'final_dataset.csv'")

Final dataset has been saved to 'final_dataset.csv'


In [22]:
po=pd.read_csv(r"C:\Users\aditya\Downloads\readmission\mywork\cleanednew\final_dataset3.csv")

In [24]:
po

,Unnamed: 0,row_id_x,subject_id,hadm_id,icd9_code,admittime,dischtime,admission_location,discharge_location,hospital_expire_flag,length_of_stay,row_id_y,gender,age,abnormal_lab_count,lab_test_count,procedure_count,readmitted_30_days
0,33,1493,113,109976,1915,2140-12-12 22:46:00,2140-12-27 19:30:00,EMERGENCY ROOM ADMIT,HOME,0,14,104,M,100,27.0,194.0,2.0,0
1,34,1494,113,109976,3314,2140-12-12 22:46:00,2140-12-27 19:30:00,EMERGENCY ROOM ADMIT,HOME,0,14,104,M,100,27.0,194.0,2.0,0
2,35,1495,113,109976,53081,2140-12-12 22:46:00,2140-12-27 19:30:00,EMERGENCY ROOM ADMIT,HOME,0,14,104,M,100,27.0,194.0,2.0,0
3,36,1496,114,178393,41401,2146-08-29 01:18:00,2146-09-03 16:06:00,PHYS REFERRAL/NORMAL DELI,HOME,0,5,105,M,100,21.0,61.0,3.0,0
4,37,1497,114,178393,4111,2146-08-29 01:18:00,2146-09-03 16:06:00,PHYS REFERRAL/NORMAL DELI,HOME,0,5,105,M,100,21.0,61.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357276,651036,639792,97497,168949,27669,2106-07-16 14:52:00,2106-07-24 14:15:00,CLINIC REFERRAL/PREMATURE,HOME,0,7,45870,M,100,47.0,175.0,3.0,0
357277,651037,639793,97497,168949,5178,2106-07-16 14:52:00,2106-07-24 14:15:00,CLINIC REFERRAL/PREMATURE,HOME,0,7,45870,M,100,47.0,175.0,3.0,0
357278,651038,639794,97497,168949,42731,2106-07-16 14:52:00,2106-07-24 14:15:00,CLINIC REFERRAL/PREMATURE,HOME,0,7,45870,M,100,47.0,175.0,3.0,0
357279,651039,639795,97497,168949,V5861,2106-07-16 14:52:00,2106-07-24 14:15:00,CLINIC REFERRAL/PREMATURE,HOME,0,7,45870,M,100,47.0,175.0,3.0,0


In [ ]:
# diagnoses_icd = pd.read_csv(r"C:\Users\aditya\Downloads\readmission\mywork\cleanednew\diagnoses_icd_modified.csv")
# admissions = pd.read_csv(r"C:\Users\aditya\Downloads\readmission\mywork\cleanednew\admissions_filtered.csv")
# patients = pd.read_csv(r"C:\Users\aditya\Downloads\readmission\mywork\cleanednew\patients_modified.csv")
# labevents = pd.read_csv(r"C:\Users\aditya\Downloads\readmission\mywork\cleanednew\labevents_modified.csv")
# procedures_icd = pd.read_csv(r'C:\Users\aditya\Downloads\readmission\mywork\cleanednew\procedures_icd_modified2.csv')

,row_id,subject_id,hadm_id,icd9_code
0,1511,115,114585,4280
1,1527,117,140784,4280
2,1610,124,138376,42833
3,1613,124,138376,4280
4,1671,130,198214,4280
...,...,...,...,...
21269,638387,97144,109999,4280
21270,638481,97172,133092,42833
21271,638485,97172,133092,4280
21272,639751,97488,152542,4280


In [25]:
po.drop(columns=["hospital_expire_flag","admission_location","row_id_y","procedure_count","row_id_x","age"],inplace=True)

In [28]:
po

,Unnamed: 0,subject_id,hadm_id,icd9_code,admittime,dischtime,discharge_location,length_of_stay,gender,abnormal_lab_count,lab_test_count,readmitted_30_days
0,33,113,109976,1915,2140-12-12 22:46:00,2140-12-27 19:30:00,HOME,14,M,27.0,194.0,0
1,34,113,109976,3314,2140-12-12 22:46:00,2140-12-27 19:30:00,HOME,14,M,27.0,194.0,0
2,35,113,109976,53081,2140-12-12 22:46:00,2140-12-27 19:30:00,HOME,14,M,27.0,194.0,0
3,36,114,178393,41401,2146-08-29 01:18:00,2146-09-03 16:06:00,HOME,5,M,21.0,61.0,0
4,37,114,178393,4111,2146-08-29 01:18:00,2146-09-03 16:06:00,HOME,5,M,21.0,61.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
357276,651036,97497,168949,27669,2106-07-16 14:52:00,2106-07-24 14:15:00,HOME,7,M,47.0,175.0,0
357277,651037,97497,168949,5178,2106-07-16 14:52:00,2106-07-24 14:15:00,HOME,7,M,47.0,175.0,0
357278,651038,97497,168949,42731,2106-07-16 14:52:00,2106-07-24 14:15:00,HOME,7,M,47.0,175.0,0
357279,651039,97497,168949,V5861,2106-07-16 14:52:00,2106-07-24 14:15:00,HOME,7,M,47.0,175.0,0


In [29]:
po["discharge_location"].value_counts()

discharge_location
HOME                         113892
HOME HEALTH CARE             103937
SNF                           51782
REHAB/DISTINCT PART HOSP      41623
LONG TERM CARE HOSPITAL       23127
SHORT TERM HOSPITAL           10849
DISCH-TRAN TO PSYCH HOSP       4792
LEFT AGAINST MEDICAL ADVI      2326
DISC-TRAN CANCER/CHLDRN H      2185
HOSPICE-HOME                   1000
OTHER FACILITY                  675
ICF                             455
HOSPICE-MEDICAL FACILITY        295
HOME WITH HOME IV PROVIDR       261
DISC-TRAN TO FEDERAL HC          82
Name: count, dtype: int64

In [30]:
po['gender'].replace({'M': 1, 'F': 0}, inplace=True)

In [31]:
po

,Unnamed: 0,subject_id,hadm_id,icd9_code,admittime,dischtime,discharge_location,length_of_stay,gender,abnormal_lab_count,lab_test_count,readmitted_30_days
0,33,113,109976,1915,2140-12-12 22:46:00,2140-12-27 19:30:00,HOME,14,1,27.0,194.0,0
1,34,113,109976,3314,2140-12-12 22:46:00,2140-12-27 19:30:00,HOME,14,1,27.0,194.0,0
2,35,113,109976,53081,2140-12-12 22:46:00,2140-12-27 19:30:00,HOME,14,1,27.0,194.0,0
3,36,114,178393,41401,2146-08-29 01:18:00,2146-09-03 16:06:00,HOME,5,1,21.0,61.0,0
4,37,114,178393,4111,2146-08-29 01:18:00,2146-09-03 16:06:00,HOME,5,1,21.0,61.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
357276,651036,97497,168949,27669,2106-07-16 14:52:00,2106-07-24 14:15:00,HOME,7,1,47.0,175.0,0
357277,651037,97497,168949,5178,2106-07-16 14:52:00,2106-07-24 14:15:00,HOME,7,1,47.0,175.0,0
357278,651038,97497,168949,42731,2106-07-16 14:52:00,2106-07-24 14:15:00,HOME,7,1,47.0,175.0,0
357279,651039,97497,168949,V5861,2106-07-16 14:52:00,2106-07-24 14:15:00,HOME,7,1,47.0,175.0,0


In [32]:
po.drop(columns=["admittime","dischtime"],inplace=True)

In [33]:
po

,Unnamed: 0,subject_id,hadm_id,icd9_code,discharge_location,length_of_stay,gender,abnormal_lab_count,lab_test_count,readmitted_30_days
0,33,113,109976,1915,HOME,14,1,27.0,194.0,0
1,34,113,109976,3314,HOME,14,1,27.0,194.0,0
2,35,113,109976,53081,HOME,14,1,27.0,194.0,0
3,36,114,178393,41401,HOME,5,1,21.0,61.0,0
4,37,114,178393,4111,HOME,5,1,21.0,61.0,0
...,...,...,...,...,...,...,...,...,...,...
357276,651036,97497,168949,27669,HOME,7,1,47.0,175.0,0
357277,651037,97497,168949,5178,HOME,7,1,47.0,175.0,0
357278,651038,97497,168949,42731,HOME,7,1,47.0,175.0,0
357279,651039,97497,168949,V5861,HOME,7,1,47.0,175.0,0


In [ ]:
import pandas as pd

admissions_df = po

categories_to_replace = [
    'HOSPICE-MEDICAL FACILITY', 'ICF', 'DISCH-TRAN TO PSYCH HOSP', 
    'HOME WITH HOME IV PROVIDR', 'DISC-TRAN TO FEDERAL HC'
]
admissions_df['discharge_location'] = admissions_df['discharge_location'].replace(categories_to_replace, 'OTHER FACILITY')

discharge_mapping = {
    'HOME HEALTH CARE': 1,
    'SNF': 2,
    'HOME': 3,
    'REHAB/DISTINCT PART HOSP': 4,
    'LONG TERM CARE HOSPITAL': 5,
    'SHORT TERM HOSPITAL': 6,
    'LEFT AGAINST MEDICAL ADVI': 7,
    'HOSPICE-HOME': 8,
    'DISC-TRAN CANCER/CHLDRN H': 9,
    'OTHER FACILITY': 10
}
admissions_df['discharge_location'] = admissions_df['discharge_location'].map(discharge_mapping)

In [ ]:
# admissions_df.drop(columns=["Unnamed:0"],inplace=True)

KeyError: "['Unnamed:0'] not found in axis"

In [41]:
admissions_df = admissions_df.drop(columns=['Unnamed: 0'])

In [42]:
admissions_df

,icd9_code,discharge_location,length_of_stay,gender,abnormal_lab_count,lab_test_count,readmitted_30_days
0,1915,3,14,1,27.0,194.0,0
1,3314,3,14,1,27.0,194.0,0
2,53081,3,14,1,27.0,194.0,0
3,41401,3,5,1,21.0,61.0,0
4,4111,3,5,1,21.0,61.0,0
...,...,...,...,...,...,...,...
357276,27669,3,7,1,47.0,175.0,0
357277,5178,3,7,1,47.0,175.0,0
357278,42731,3,7,1,47.0,175.0,0
357279,V5861,3,7,1,47.0,175.0,0


In [43]:
admissions_df.to_csv(r"C:\Users\aditya\Downloads\readmission\mywork\cleanednew\final_dataset3.csv")

In [44]:
admissions_df

,icd9_code,discharge_location,length_of_stay,gender,abnormal_lab_count,lab_test_count,readmitted_30_days
0,1915,3,14,1,27.0,194.0,0
1,3314,3,14,1,27.0,194.0,0
2,53081,3,14,1,27.0,194.0,0
3,41401,3,5,1,21.0,61.0,0
4,4111,3,5,1,21.0,61.0,0
...,...,...,...,...,...,...,...
357276,27669,3,7,1,47.0,175.0,0
357277,5178,3,7,1,47.0,175.0,0
357278,42731,3,7,1,47.0,175.0,0
357279,V5861,3,7,1,47.0,175.0,0


In [47]:
admissions_df["icd9_code"].value_counts()

icd9_code
4019     12370
41401     7425
2724      5935
V053      5777
42731     5601
         ...  
0931         1
1348         1
63590        1
7753         1
6940         1
Name: count, Length: 6422, dtype: int64

In [48]:
# Show all ICD-9 code counts
pd.set_option('display.max_rows', None)  # Show all rows
print(admissions_df["icd9_code"].value_counts())
pd.reset_option('display.max_rows')  # Reset if needed


icd9_code
4019     12370
41401     7425
2724      5935
V053      5777
42731     5601
V290      5494
4280      5091
25000     4636
53081     4071
5849      3687
2720      3624
V3000     3554
2859      3070
5990      2786
V3001     2734
51881     2683
2851      2600
2449      2435
3051      2403
7742      2239
311       2205
V502      2016
2762      1978
V5861     1891
486       1849
412       1742
496       1706
V1582     1680
40390     1653
4240      1620
32723     1614
V4582     1581
49390     1543
4241      1540
5859      1526
V5867     1504
2761      1456
9971      1454
42789     1441
2875      1413
41071     1398
45829     1359
5070      1357
99592     1290
769       1274
5119      1267
V4581     1250
4111      1228
5180      1208
0389      1171
27800     1130
30000     1087
7793      1064
2749      1043
4168      1041
77081     1041
73300     1027
V3101      984
4589       967
78039      947
2767       941
76518      922
E8782      881
2760       880
76527      880
76528      871


In [ ]:
import pandas as pd

df = admissions_df
df['icd9_code'] = df['icd9_code'].astype(str)

df_numeric = df[df['icd9_code'].str.match(r'^\d+$')]

code_counts = df_numeric['icd9_code'].value_counts()

df_filtered = df_numeric[df_numeric['icd9_code'].isin(code_counts[code_counts >= 30].index)]

print(f"Original DataFrame shape: {df.shape}")
print(f"Shape after removing alphabetic icd9_codes: {df_numeric.shape}")
print(f"Shape after removing icd9_codes with count < 30: {df_filtered.shape}")

print("\nFirst few rows of the filtered DataFrame:")
print(df_filtered.head(10))


Original DataFrame shape: (357281, 7)
Shape after removing alphabetic icd9_codes: (294588, 7)
Shape after removing icd9_codes with count < 30: (268129, 7)

First few rows of the filtered DataFrame:
   icd9_code  discharge_location  length_of_stay  gender  abnormal_lab_count  \
1       3314                   3              14       1                27.0   
2      53081                   3              14       1                27.0   
3      41401                   3               5       1                21.0   
4       4111                   3               5       1                21.0   
5      48283                   3               5       1                21.0   
6       2859                   3               5       1                21.0   
7       2720                   3               5       1                21.0   
8       3051                   3               5       1                21.0   
21     30590                   9               1       1                 8.0   
28

In [52]:
df_filtered["icd9_code"].value_counts()

icd9_code
4019     12370
41401     7425
2724      5935
42731     5601
4280      5091
         ...  
4162        30
7767        30
7469        30
78909       30
25022       30
Name: count, Length: 1080, dtype: int64

In [ ]:
df_filtered["icd9_code"].unique()

1080

In [61]:
df_filtered

,icd9_code,discharge_location,length_of_stay,gender,abnormal_lab_count,lab_test_count,readmitted_30_days
1,3314,3,14,1,27.0,194.0,0
2,53081,3,14,1,27.0,194.0,0
3,41401,3,5,1,21.0,61.0,0
4,4111,3,5,1,21.0,61.0,0
5,48283,3,5,1,21.0,61.0,0
...,...,...,...,...,...,...,...
357275,135,3,7,1,47.0,175.0,0
357276,27669,3,7,1,47.0,175.0,0
357277,5178,3,7,1,47.0,175.0,0
357278,42731,3,7,1,47.0,175.0,0


In [ ]:
import pandas as pd

severity_ranking = {
    '4289': 1,   # Heart failure, unspecified
    '4280': 2,   # Congestive heart failure, unspecified
    '4281': 3,   # Left heart failure
    '40291': 4,  # Hypertensive heart disease, unspecified, with HF
    '40211': 5,  # Hypertensive heart disease, benign, with HF
    '42820': 6,  # Systolic heart failure, unspecified
    '42830': 6,  # Diastolic heart failure, unspecified
    '42821': 7,  # Systolic heart failure, acute
    '42831': 7,  # Diastolic heart failure, acute
    '42822': 8,  # Systolic heart failure, chronic
    '42832': 8,  # Diastolic heart failure, chronic
    '42823': 9,  # Systolic heart failure, acute on chronic
    '42833': 9,  # Diastolic heart failure, acute on chronic
    '42840': 10, # Combined systolic/diastolic heart failure, unspecified
    '42841': 11, # Combined systolic/diastolic heart failure, acute
    '42842': 12, # Combined systolic/diastolic heart failure, chronic
    '42843': 13, # Combined systolic/diastolic heart failure, acute on chronic
    '40201': 14, # Hypertensive heart disease, malignant, with HF
    '39891': 15, # Rheumatic heart failure
    '40491': 16, # Hypertensive heart and renal disease, unspecified, with HF
    '40493': 16, # Hypertensive heart and renal disease, unspecified, with HF and CKD
    '40411': 17, # Hypertensive heart and renal disease, benign, with HF
    '40413': 17, # Hypertensive heart and renal disease, benign, with HF and CKD
    '40401': 18, # Hypertensive heart and renal disease, malignant, with HF
    '40403': 18  # Hypertensive heart and renal disease, malignant, with HF and CKD
}

adjusted_severity_ranking = {code: rank + 1 for code, rank in severity_ranking.items()}
df_filtered['icd9_code'] = df_filtered['icd9_code'].astype(str)

df_filtered['severity_rank'] = df_filtered['icd9_code'].map(adjusted_severity_ranking).fillna(1).astype(int)

print("First few rows of the DataFrame with severity ranks:")
print(df_filtered.head(10))

print("\nUnique ICD-9 codes and their severity ranks:")
print(df_filtered[['icd9_code', 'severity_rank']])


First few rows of the DataFrame with severity ranks:
   icd9_code  discharge_location  length_of_stay  gender  abnormal_lab_count  \
1       3314                   3              14       1                27.0   
2      53081                   3              14       1                27.0   
3      41401                   3               5       1                21.0   
4       4111                   3               5       1                21.0   
5      48283                   3               5       1                21.0   
6       2859                   3               5       1                21.0   
7       2720                   3               5       1                21.0   
8       3051                   3               5       1                21.0   
21     30590                   9               1       1                 8.0   
28     41011                   3               3       1                15.0   

    lab_test_count  readmitted_30_days  severity_rank  
1         

C:\Users\aditya\AppData\Local\Temp\ipykernel_23900\2753781644.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['icd9_code'] = df_filtered['icd9_code'].astype(str)
C:\Users\aditya\AppData\Local\Temp\ipykernel_23900\2753781644.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['severity_rank'] = df_filtered['icd9_code'].map(adjusted_severity_ranking).fillna(1).astype(int)


In [65]:
df_filtered.to_csv(r"C:\Users\aditya\Downloads\readmission\mywork\cleanednew\final_dataset3.csv")

In [67]:
df_filtered.isnull().sum()

icd9_code             0
discharge_location    0
length_of_stay        0
gender                0
abnormal_lab_count    0
lab_test_count        0
readmitted_30_days    0
severity_rank         0
dtype: int64

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

df = df_filtered

df = df.drop(columns=['icd9_code'])

print("Missing values before cleaning:\n", df.isna().sum())

numerical_cols = ['length_of_stay', 'lab_test_count', 'abnormal_lab_count']

categorical_cols = ['discharge_location', 'gender', 'readmitted_30_days', 'severity_rank']
df = df.dropna(subset=categorical_cols)

for col in categorical_cols:
    df[col] = df[col].astype(int)

for col in numerical_cols:
    df[col] = df[col].astype(float)

def cap_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
    return df

for col in numerical_cols:
    df[col] = df[col].clip(lower=0)
    df = cap_outliers(df, col)

# Step 6: Create two copies of the DataFrame for different scalers
df_minmax = df.copy()
df_standard = df.copy()

# Step 7: Normalize numerical columns using MinMaxScaler
minmax_scaler = MinMaxScaler()
df_minmax[numerical_cols] = minmax_scaler.fit_transform(df_minmax[numerical_cols])

# Step 8: Standardize numerical columns using StandardScaler
standard_scaler = StandardScaler()
df_standard[numerical_cols] = standard_scaler.fit_transform(df_standard[numerical_cols])


# Step 10: Save the cleaned and scaled datasets
output_path_minmax = r'C:\Users\aditya\Downloads\readmission\mywork\cleanednew\finaldatset3aftericd9_minmax.csv'
output_path_standard = r'C:\Users\aditya\Downloads\readmission\mywork\cleanednew\finaldatset3aftericd9done_standard.csv'

df_minmax.to_csv(output_path_minmax, index=False)
df_standard.to_csv(output_path_standard, index=False)

print(f"Cleaned and MinMax-scaled dataset saved to '{output_path_minmax}'")
print(f"Cleaned and Standard-scaled dataset saved to '{output_path_standard}'")

# Display dataset info and summary statistics for both versions
print("\nMinMax-scaled Dataset info:\n")
print(df_minmax.info())
print("\nMinMax-scaled Summary statistics:\n", df_minmax.describe())

print("\nStandard-scaled Dataset info:\n")
print(df_standard.info())
print("\nStandard-scaled Summary statistics:\n", df_standard.describe())

Missing values before cleaning:
 discharge_location    0
length_of_stay        0
gender                0
abnormal_lab_count    0
lab_test_count        0
readmitted_30_days    0
severity_rank         0
dtype: int64
Cleaned and MinMax-scaled dataset saved to 'C:\Users\aditya\Downloads\readmission\mywork\cleanednew\finaldatset3aftericd9_minmax.csv'
Cleaned and Standard-scaled dataset saved to 'C:\Users\aditya\Downloads\readmission\mywork\cleanednew\finaldatset3aftericd9done_standard.csv'

MinMax-scaled Dataset info:

<class 'pandas.core.frame.DataFrame'>
Index: 268129 entries, 1 to 357280
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   discharge_location  268129 non-null  int32  
 1   length_of_stay      268129 non-null  float64
 2   gender              268129 non-null  int32  
 3   abnormal_lab_count  268129 non-null  float64
 4   lab_test_count      268129 non-null  float64
 5   readmitted_30_days  

In [70]:
df_minmax

,discharge_location,length_of_stay,gender,abnormal_lab_count,lab_test_count,readmitted_30_days,severity_rank
1,3,0.528302,1,0.136020,0.350814,0,1
2,3,0.528302,1,0.136020,0.350814,0,1
3,3,0.188679,1,0.105793,0.110307,0,1
4,3,0.188679,1,0.105793,0.110307,0,1
5,3,0.188679,1,0.105793,0.110307,0,1
...,...,...,...,...,...,...,...
357275,3,0.264151,1,0.236776,0.316456,0,1
357276,3,0.264151,1,0.236776,0.316456,0,1
357277,3,0.264151,1,0.236776,0.316456,0,1
357278,3,0.264151,1,0.236776,0.316456,0,1


In [71]:
df_standard

,discharge_location,length_of_stay,gender,abnormal_lab_count,lab_test_count,readmitted_30_days,severity_rank
1,3,0.547167,1,-0.651621,-0.039162,0,1
2,3,0.547167,1,-0.651621,-0.039162,0,1
3,3,-0.635467,1,-0.752350,-0.879794,0,1
4,3,-0.635467,1,-0.752350,-0.879794,0,1
5,3,-0.635467,1,-0.752350,-0.879794,0,1
...,...,...,...,...,...,...,...
357275,3,-0.372659,1,-0.315859,-0.159253,0,1
357276,3,-0.372659,1,-0.315859,-0.159253,0,1
357277,3,-0.372659,1,-0.315859,-0.159253,0,1
357278,3,-0.372659,1,-0.315859,-0.159253,0,1
